Setting up PySpark

In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("Practise").getOrCreate()

# Verify Spark session creation
print(spark)

your 131072x1 screen size is bogus. expect trouble
24/10/15 16:52:07 WARN Utils: Your hostname, 590-mxl3363mnx resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/15 16:52:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/15 16:52:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/15 16:52:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Loading Data

In [38]:
df_pyspark =  spark.read.csv('Placement_Data_Full_Class.csv',header =True ,inferSchema = True)
df_pyspark.show()

+-----+------+-----+-------+-----+-------+--------+--------+---------+------+-------+--------------+-----+----------+------+
|sl_no|gender|ssc_p|  ssc_b|hsc_p|  hsc_b|   hsc_s|degree_p| degree_t|workex|etest_p|specialisation|mba_p|    status|salary|
+-----+------+-----+-------+-----+-------+--------+--------+---------+------+-------+--------------+-----+----------+------+
|    1|     M| 67.0| Others| 91.0| Others|Commerce|    58.0| Sci&Tech|    No|   55.0|        Mkt&HR| 58.8|    Placed|270000|
|    2|     M|79.33|Central|78.33| Others| Science|   77.48| Sci&Tech|   Yes|   86.5|       Mkt&Fin|66.28|    Placed|200000|
|    3|     M| 65.0|Central| 68.0|Central|    Arts|    64.0|Comm&Mgmt|    No|   75.0|       Mkt&Fin| 57.8|    Placed|250000|
|    4|     M| 56.0|Central| 52.0|Central| Science|    52.0| Sci&Tech|    No|   66.0|        Mkt&HR|59.43|Not Placed|  NULL|
|    5|     M| 85.8|Central| 73.6|Central|Commerce|    73.3|Comm&Mgmt|    No|   96.8|       Mkt&Fin| 55.5|    Placed|425000|


Dropping Null Values

In [39]:
df_pyspark = df_pyspark.na.drop()

In [40]:
df_pyspark.printSchema()

root
 |-- sl_no: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- ssc_p: double (nullable = true)
 |-- ssc_b: string (nullable = true)
 |-- hsc_p: double (nullable = true)
 |-- hsc_b: string (nullable = true)
 |-- hsc_s: string (nullable = true)
 |-- degree_p: double (nullable = true)
 |-- degree_t: string (nullable = true)
 |-- workex: string (nullable = true)
 |-- etest_p: double (nullable = true)
 |-- specialisation: string (nullable = true)
 |-- mba_p: double (nullable = true)
 |-- status: string (nullable = true)
 |-- salary: integer (nullable = true)



### Data Preprocessing

[gender,ssc_p,specilisation]------>new feature ------> independent feature

In [57]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=["ssc_p","hsc_p","degree_p",
                                              "etest_p","mba_p"],outputCol="Independent Features")

In [42]:
output = featureassembler.transform(df_pyspark)

In [43]:
output.show()

+-----+------+-----+-------+-----+-------+--------+--------+---------+------+-------+--------------+-----+------+------+--------------------+
|sl_no|gender|ssc_p|  ssc_b|hsc_p|  hsc_b|   hsc_s|degree_p| degree_t|workex|etest_p|specialisation|mba_p|status|salary|Independent Features|
+-----+------+-----+-------+-----+-------+--------+--------+---------+------+-------+--------------+-----+------+------+--------------------+
|    1|     M| 67.0| Others| 91.0| Others|Commerce|    58.0| Sci&Tech|    No|   55.0|        Mkt&HR| 58.8|Placed|270000|[67.0,91.0,58.0,5...|
|    2|     M|79.33|Central|78.33| Others| Science|   77.48| Sci&Tech|   Yes|   86.5|       Mkt&Fin|66.28|Placed|200000|[79.33,78.33,77.4...|
|    3|     M| 65.0|Central| 68.0|Central|    Arts|    64.0|Comm&Mgmt|    No|   75.0|       Mkt&Fin| 57.8|Placed|250000|[65.0,68.0,64.0,7...|
|    5|     M| 85.8|Central| 73.6|Central|Commerce|    73.3|Comm&Mgmt|    No|   96.8|       Mkt&Fin| 55.5|Placed|425000|[85.8,73.6,73.3,9...|
|    8

In [44]:
output.columns

['sl_no',
 'gender',
 'ssc_p',
 'ssc_b',
 'hsc_p',
 'hsc_b',
 'hsc_s',
 'degree_p',
 'degree_t',
 'workex',
 'etest_p',
 'specialisation',
 'mba_p',
 'status',
 'salary',
 'Independent Features']

In [45]:
finalized_data=output.select("Independent Features","salary")

In [46]:
finalized_data.show()

+--------------------+------+
|Independent Features|salary|
+--------------------+------+
|[67.0,91.0,58.0,5...|270000|
|[79.33,78.33,77.4...|200000|
|[65.0,68.0,64.0,7...|250000|
|[85.8,73.6,73.3,9...|425000|
|[82.0,64.0,66.0,6...|252000|
|[73.0,79.0,72.0,9...|231000|
|[58.0,61.0,60.0,6...|260000|
|[69.6,68.4,78.3,6...|250000|
|[77.0,87.0,59.0,6...|218000|
|[65.0,75.0,69.0,7...|200000|
|[63.0,66.2,65.6,6...|300000|
|[60.0,67.0,70.0,5...|236000|
|[62.0,65.0,66.0,5...|265000|
|[79.0,76.0,85.0,9...|393000|
|[69.8,60.8,72.23,...|360000|
|[77.4,60.0,64.74,...|300000|
|[76.5,97.7,78.86,...|360000|
|[71.0,79.0,66.0,9...|240000|
|[63.0,67.0,66.0,6...|265000|
|[76.76,76.5,67.5,...|350000|
+--------------------+------+
only showing top 20 rows



Splitting Data

In [50]:
from pyspark.ml.regression import LinearRegression

#train test split
train_data,test_data = finalized_data.randomSplit([0.60,0.40])



24/10/16 08:09:59 WARN Instrumentation: [d53d8768] regParam is zero, which might cause numerical instability and overfitting.


Building and Training the Model

In [59]:
regressor=LinearRegression(featuresCol='Independent Features', labelCol='salary')
reg=regressor.fit(train_data)

24/10/16 08:17:55 WARN Instrumentation: [c5614fee] regParam is zero, which might cause numerical instability and overfitting.


In [51]:
train_data.show()

+--------------------+------+
|Independent Features|salary|
+--------------------+------+
|[49.0,59.0,65.0,8...|340000|
|[53.0,63.0,60.0,7...|250000|
|[55.68,61.33,56.8...|260000|
|[58.0,61.0,60.0,6...|260000|
|[58.0,61.0,61.0,5...|250000|
|[58.0,62.0,64.0,5...|260000|
|[60.23,69.0,66.0,...|230000|
|[60.4,66.6,65.0,7...|220000|
|[60.8,68.4,64.6,8...|940000|
|[61.0,82.0,69.0,8...|300000|
|[62.0,62.0,60.0,6...|240000|
|[62.0,63.0,64.0,6...|220000|
|[62.0,63.0,66.0,8...|233000|
|[62.0,72.0,65.0,6...|216000|
|[63.0,62.0,68.0,6...|250000|
|[63.0,66.2,65.6,6...|300000|
|[63.0,67.0,64.0,7...|500000|
|[63.0,67.0,66.0,6...|265000|
|[63.0,72.0,68.0,7...|380000|
|[64.0,73.5,73.0,5...|250000|
+--------------------+------+
only showing top 20 rows



In [53]:
##Coefficients 

reg.coefficients

DenseVector([-442.2366, -1667.2487, -2661.5742, 1148.646, 4469.8797])

In [54]:
##Intercepts

reg.intercept

251062.8294730167

In [55]:
##Predictions

pred_results = reg.evaluate(test_data)

In [56]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|salary|        prediction|
+--------------------+------+------------------+
|[52.6,65.58,72.11...|265000|245962.29959043805|
|[56.6,64.8,70.2,8...|336000|328324.33526000456|
|[58.0,60.0,72.0,7...|275000|258419.60323793034|
|[59.0,60.0,56.0,5...|220000|  297869.364187403|
|[60.0,63.0,56.0,8...|300000| 315464.7845681075|
|[60.0,67.0,70.0,5...|236000|332655.36414322973|
|[61.0,62.0,65.0,6...|250000| 272864.5794460012|
|[61.0,81.0,66.4,5...|278000|248836.54379706466|
|[62.0,65.0,66.0,5...|265000|  250483.583764381|
|[63.0,60.0,57.0,7...|204000| 304883.6049677954|
|[63.0,71.4,61.4,6...|240000|317793.20039828395|
|[64.0,67.0,69.6,5...|250000| 309305.2901696431|
|[64.0,80.0,65.0,6...|500000|253322.61436181236|
|[65.0,68.0,69.0,5...|250000| 232866.2990523775|
|[65.0,71.5,62.8,5...|265000|254430.32899338292|
|[66.5,70.4,71.93,...|230000|  270179.337114915|
|[67.0,63.0,72.0,5...|225000|259112.58319930016|
|[67.0,67.0,73.0,5..

Model Evaluation


predictions = reg.transform(test_data)
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol='Independent Features',predictionCol='salary',metricName='rmse')
rmse = reg.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data",rmse)

new_data = spark.read.csv("path_to_new_data.csv", header=True, inferSchema=True)
new_data = assembler.transform(new_data)
predictions = model.transform(new_data)